# Data Cleaning Listado de Listado de Beneficiarios 2019

## Autores
- José Luis Delgado Dávara
- Arturo Ortiz Aguilar
- Beltrán Valle Gutiérrez-Cortines

## Importante leer para entender

En este Notebook se trabaja con 3 listados importantes:

1. beneficiarios_19 -> Dataset con el listado de TODOS los beneficiarios de 2019.
2. Estados_Beneficiarios_2019_2022 -> Dataset sólo con los estados *únicos* encontrados en el dataset anterior.
3. Diccionario -> Emparejamiento entre ambos listados de beneficiarios
4. Dataset_Inegi -> Catálogo obtenido de Inegi

In [1]:
import pandas as pd
import seaborn as sns
from thefuzz import fuzz
from thefuzz import process
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import re
import unidecode

In [2]:
def clean_text(text):
    """
    De esta manera tenemos el texto sin espacios blancos extra y sobre todo con todas las palabras con capitalización correcta.
    """
    if pd.isna(text):
        return text
    text = text.strip()  # Eliminate white spaces
    text = text.lower()  # Convert to lowercase
    text = unidecode.unidecode(text)  # Remove accents
    text = re.sub('-.*-', '', text) #Remove what is in between - -
    text = re.sub('\s+', ' ', text)  # Eliminate extra white spaces
    text = re.sub('^\s+|\s+?$', '', text)  # Eliminate spaces at the beginning and end
    return text

# 1. Lectura y limpieza de datos

### 1.1 Beneficiarios 2019

Obtenemos el listado único de localidades y listado único de municipios con sus claves.

In [3]:
beneficiarios_19 = pd.read_csv('../../data/productores_beneficiarios 2019-2022/fertilizantes_2019.csv', encoding='utf-8', skiprows=0)

In [4]:
beneficiarios_19 = beneficiarios_19[~((beneficiarios_19['ENTIDAD'] == 'NACIONAL') & (beneficiarios_19['MUNICIPIO'] == 'NACIONAL') & (beneficiarios_19['LOCALIDAD'] == 'NACIONAL'))]

In [5]:
beneficiarios_19.shape

(275026, 16)

In [6]:
beneficiarios_19['ENTIDAD'].unique()

array(['GUERRERO', nan], dtype=object)

Dropeamos los tres valores nulos debido a que las rows enteras salen como nan

In [7]:
beneficiarios_19.dropna(inplace=True)

In [8]:
# Obtenemos las localidades únicas en el dataset.
Municipios_19 = beneficiarios_19[['ENTIDAD', 'MUNICIPIO']]
Municipios_19 = Municipios_19.drop_duplicates()

In [9]:
# Estandarizamos la limpieza de los datos
Municipios_19['ENTIDAD_c_benef'] = Municipios_19['ENTIDAD'].apply(clean_text)
Municipios_19['MUNICIPIO_c_benef'] = Municipios_19['MUNICIPIO'].apply(clean_text)

In [10]:
# Creamos las dos keys de beneficiarios 
Municipios_19['KEY_benef_mun'] = Municipios_19['ENTIDAD_c_benef'].astype(str) + '-' + Municipios_19[
    'MUNICIPIO_c_benef'].astype(str)

In [11]:
Municipios_19.shape

(81, 5)

In [12]:
Municipios_19.columns

Index(['ENTIDAD', 'MUNICIPIO', 'ENTIDAD_c_benef', 'MUNICIPIO_c_benef',
       'KEY_benef_mun'],
      dtype='object')

### 1.2 INEGI 2019

Obtener listado único de municipios y listado único de localidades de inegi de 2019 con sus claves.

In [13]:
path_dataset_inegi_2019 = '../../data/inegi/dataset_inegi_clean_2019.csv'
#dataset_inegi_clean = pd.read_csv(path_dataset_inegi_2022, encoding='utf-8', dtype={'CVE_ENT': str, 'CVE_MUN': str, 'CVE_LOC': str})
dataset_inegi_2019 = pd.read_csv(path_dataset_inegi_2019)

### 1.2.1 INEGI 2019 Municipios únicos para cada año.

In [14]:
dataset_inegi_2019['KEY_inegi_municipio'] = dataset_inegi_2019['Entidad_c_inegi'].astype(str) + '-' + dataset_inegi_2019[
    'Municipio_c_inegi'].astype(str) 
dataset_inegi_2019['KEY_inegi_localidad'] = dataset_inegi_2019['Municipio_c_inegi'].astype(str) + '-' + dataset_inegi_2019['Localidad_c_inegi'].astype(str)

In [15]:
INEGI_UNIQUEMUN_2019 = dataset_inegi_2019.drop(columns=["CVE_LOC", "Localidad_inegi", "Localidad_c_inegi", "KEY_inegi_localidad", "POB_TOTAL"])

INEGI_UNIQUEMUN_2019 = INEGI_UNIQUEMUN_2019.drop_duplicates()

In [16]:
INEGI_UNIQUEMUN_2019.shape

(2465, 7)

In [17]:
INEGI_UNIQUEMUN_2019.columns

Index(['CVE_ENT', 'Entidad_inegi', 'CVE_MUN', 'Municipio_inegi',
       'Entidad_c_inegi', 'Municipio_c_inegi', 'KEY_inegi_municipio'],
      dtype='object')

# 2. Diccionario de los datasets de INEGI Y LISTADO BENEFICIARIOS 2019

El objetivo de esta sección es crear dos diccionarios de códigos según BENEFICIARIOS-MUNICIPIOS_INEGI y otro BENEFICIARIOS-LOCALIDADES_INEGI, para cada uno de los estados encontrados en inegi_2019, en este caso el número de keys es menor con lo cual no nos hará falta dividir por estado.

Para ello haremos un Left join entre Localidades_19 y el dataset de INEGI correspondiente.

In [18]:
# Crear una función para encontrar la mejor coincidencia difusa con límites entre 85 y 100 de coincidencia
def fuzzy_merge_benef2019_2022(df_benef, df_inegi, key1, key2, threshold=85, limit=1):
    """
    df_inegi: DataFrame de la izquierda (el DataFrame principal)
    df_prod: DataFrame de la derecha (el DataFrame con el que se quiere hacer el join)
    key1: Columna de la clave en df_inegi
    key2: Columna de la clave en df_prod
    threshold: Umbral de coincidencia difusa
    limit: Número de coincidencias a encontrar
    """
    s = df_inegi[key2].tolist()
    
    # Encontrar las mejores coincidencias para cada clave en df_inegi
    matches = df_benef[key1].apply(lambda x: process.extractOne(x, s, score_cutoff=threshold))


    # Crear una columna con las mejores coincidencias
    df_benef['best_match'] = [match[0] if match else None for match in matches]
    
    df_benef['match_score'] = [match[1] if match else None for match in matches]
    

    # Hacer el merge con las mejores coincidencias
    df_merged = pd.merge(df_benef, df_inegi, left_on='best_match', right_on=key2, how='left',
                         suffixes=('_benef', '_inegi'))
    
    return df_merged

In [19]:
diccionario_MUN_19 = fuzzy_merge_benef2019_2022(Municipios_19, INEGI_UNIQUEMUN_2019, 'KEY_benef_mun', 'KEY_inegi_municipio')
diccionario_MUN_19.shape

(81, 14)

In [20]:
diccionario_MUN_19.drop(columns=['ENTIDAD', 'MUNICIPIO', 'ENTIDAD_c_benef', 'MUNICIPIO_c_benef', 'Entidad_c_inegi',
       'Municipio_c_inegi'], inplace=True)

In [21]:
diccionario_MUN_19.to_csv('../../data/productores_beneficiarios 2019-2022/diccionarios_E3/diccionario_MUN_19.csv', index=False)

In [22]:
diccionario_MUN_19_simple = pd.read_csv('../../data/productores_beneficiarios 2019-2022/diccionarios_E3/diccionario_MUN_19_simple.csv')

In [23]:
nan_rows = beneficiarios_19[beneficiarios_19.isna().any(axis=1)]

In [24]:
nan_rows

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,ESTRATIFICACIÓN,PROGRAMA,COMPONENTE,SUBCOMPONENTE,PRODUCTO,FECHA,MONTO FEDERAL,APOYO,ACTIVIDAD,ESLABÓN,CICLO AGRÍCOLA


In [25]:
# Assuming your DataFrame is named df
filtered_df = beneficiarios_19[beneficiarios_19['ENTIDAD'] == 'NACIONAL']

In [26]:
filtered_df

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,ESTRATIFICACIÓN,PROGRAMA,COMPONENTE,SUBCOMPONENTE,PRODUCTO,FECHA,MONTO FEDERAL,APOYO,ACTIVIDAD,ESLABÓN,CICLO AGRÍCOLA


### MERGE

Armamos por partes el dataset definitivo:
1. Juntamos beneficiarios_19 con el diccionario simple. (listado_beneficiario_parte_I)
2. Al df anterior juntamos las claver provenientes del catálogo de Inegi. (listado_beneficiario_parte_II)

In [27]:
beneficiarios_19.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD',
       'ESTRATIFICACIÓN', 'PROGRAMA', 'COMPONENTE', 'SUBCOMPONENTE',
       'PRODUCTO', 'FECHA', 'MONTO FEDERAL', 'APOYO', 'ACTIVIDAD', 'ESLABÓN',
       'CICLO AGRÍCOLA'],
      dtype='object')

In [28]:
# Crear una variable KEY en listado de productores y el diccionario para hacer el join

# Clean listado beneficiarios
beneficiarios_19['ESTADO_Clean'] = beneficiarios_19['ENTIDAD'].apply(clean_text)
beneficiarios_19['MUNICIPIO_Clean'] = beneficiarios_19['MUNICIPIO'].apply(clean_text)

# Create KEY in listado beneficiarios
beneficiarios_19['Estado-mun-KEY'] = beneficiarios_19['ESTADO_Clean'].astype(str) + '-' + beneficiarios_19[
    'MUNICIPIO_Clean'].astype(str)

In [29]:
# Hacer el join de la Parte I
listado_beneficiarios_parte_I = pd.merge(beneficiarios_19, diccionario_MUN_19_simple, left_on="Estado-mun-KEY",
                                        right_on="KEY_benef_mun", how='left', suffixes=('_benef', '_inegi'))

In [30]:
listado_beneficiarios_parte_I.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD',
       'ESTRATIFICACIÓN', 'PROGRAMA', 'COMPONENTE', 'SUBCOMPONENTE',
       'PRODUCTO', 'FECHA', 'MONTO FEDERAL', 'APOYO', 'ACTIVIDAD', 'ESLABÓN',
       'CICLO AGRÍCOLA', 'ESTADO_Clean', 'MUNICIPIO_Clean', 'Estado-mun-KEY',
       'KEY_benef_mun', 'KEY_inegi_municipio'],
      dtype='object')

In [31]:
# Hacer el join de la Parte II
listado_beneficiarios_parte_II = pd.merge(listado_beneficiarios_parte_I, INEGI_UNIQUEMUN_2019, left_on="KEY_inegi_municipio",
                                        right_on="KEY_inegi_municipio", how='left', suffixes=('_benef', '_inegi'))

In [32]:
listado_beneficiarios_parte_II

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,ESTRATIFICACIÓN,PROGRAMA,COMPONENTE,SUBCOMPONENTE,PRODUCTO,...,MUNICIPIO_Clean,Estado-mun-KEY,KEY_benef_mun,KEY_inegi_municipio,CVE_ENT,Entidad_inegi,CVE_MUN,Municipio_inegi,Entidad_c_inegi,Municipio_c_inegi
0,TORRES ROMERO SANTANA,CENTRO PAÍS,GUERRERO,AZOYÚ,LOS CHEGÜES,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,azoyu,guerrero-azoyu,guerrero-azoyu,guerrero-azoyu,12,Guerrero,13,Azoyú,guerrero,azoyu
1,GOMEZ PADILLA DEMETRIA,CENTRO PAÍS,GUERRERO,TECOANAPA,MECATEPEC,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,tecoanapa,guerrero-tecoanapa,guerrero-tecoanapa,guerrero-tecoanapa,12,Guerrero,56,Tecoanapa,guerrero,tecoanapa
2,GUERRERO MELCHOR ADRIAN,CENTRO PAÍS,GUERRERO,TELOLOAPAN,TLAJOCOTLA (LA HACIENDA),Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,teloloapan,guerrero-teloloapan,guerrero-teloloapan,guerrero-teloloapan,12,Guerrero,58,Teloloapan,guerrero,teloloapan
3,ROMERO GONZALEZ CELERINA,CENTRO PAÍS,GUERRERO,TELOLOAPAN,TELOLOAPAN,Medio,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,teloloapan,guerrero-teloloapan,guerrero-teloloapan,guerrero-teloloapan,12,Guerrero,58,Teloloapan,guerrero,teloloapan
4,GOMEZ CERON MA.MIREYA,CENTRO PAÍS,GUERRERO,TECOANAPA,LOS SAUCES,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,tecoanapa,guerrero-tecoanapa,guerrero-tecoanapa,guerrero-tecoanapa,12,Guerrero,56,Tecoanapa,guerrero,tecoanapa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275018,CASTILLO CUEVAS AMANCIO,CENTRO PAÍS,GUERRERO,TAXCO DE ALARCÓN,PAINTLA,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,taxco de alarcon,guerrero-taxco de alarcon,guerrero-taxco de alarcon,guerrero-taxco de alarcon,12,Guerrero,55,Taxco de Alarcón,guerrero,taxco de alarcon
275019,RUEDA SECUNDINO ANACLETO,CENTRO PAÍS,GUERRERO,TAXCO DE ALARCÓN,PAINTLA,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,taxco de alarcon,guerrero-taxco de alarcon,guerrero-taxco de alarcon,guerrero-taxco de alarcon,12,Guerrero,55,Taxco de Alarcón,guerrero,taxco de alarcon
275020,ARMENTA VILLEGAS LUIS,CENTRO PAÍS,GUERRERO,COCULA,COCULA,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,cocula,guerrero-cocula,guerrero-cocula,guerrero-cocula,12,Guerrero,17,Cocula,guerrero,cocula
275021,BELLO JIMENEZ GREGORIA,CENTRO PAÍS,GUERRERO,ATLIXTAC,XALPITZÁHUAC,Muy alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,atlixtac,guerrero-atlixtac,guerrero-atlixtac,guerrero-atlixtac,12,Guerrero,10,Atlixtac,guerrero,atlixtac


In [33]:
listado_beneficiarios_parte_II.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD',
       'ESTRATIFICACIÓN', 'PROGRAMA', 'COMPONENTE', 'SUBCOMPONENTE',
       'PRODUCTO', 'FECHA', 'MONTO FEDERAL', 'APOYO', 'ACTIVIDAD', 'ESLABÓN',
       'CICLO AGRÍCOLA', 'ESTADO_Clean', 'MUNICIPIO_Clean', 'Estado-mun-KEY',
       'KEY_benef_mun', 'KEY_inegi_municipio', 'CVE_ENT', 'Entidad_inegi',
       'CVE_MUN', 'Municipio_inegi', 'Entidad_c_inegi', 'Municipio_c_inegi'],
      dtype='object')

In [34]:
listado_beneficiarios_parte_II = listado_beneficiarios_parte_II.drop(columns=['ESTADO_Clean', 'MUNICIPIO_Clean', 'Estado-mun-KEY', 'KEY_inegi_municipio', 'Entidad_c_inegi','Municipio_c_inegi', 'ESTRATIFICACIÓN', 'PROGRAMA', 'COMPONENTE', 'SUBCOMPONENTE', 'APOYO', 'ACTIVIDAD', 'ESLABÓN'])

In [35]:
listado_beneficiarios_parte_II

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,PRODUCTO,FECHA,MONTO FEDERAL,CICLO AGRÍCOLA,KEY_benef_mun,CVE_ENT,Entidad_inegi,CVE_MUN,Municipio_inegi
0,TORRES ROMERO SANTANA,CENTRO PAÍS,GUERRERO,AZOYÚ,LOS CHEGÜES,Cultivo de maíz grano,30/06/2019,2430.55,PV2019,guerrero-azoyu,12,Guerrero,13,Azoyú
1,GOMEZ PADILLA DEMETRIA,CENTRO PAÍS,GUERRERO,TECOANAPA,MECATEPEC,Cultivo de maíz grano,02/07/2019,4383.10,PV2019,guerrero-tecoanapa,12,Guerrero,56,Tecoanapa
2,GUERRERO MELCHOR ADRIAN,CENTRO PAÍS,GUERRERO,TELOLOAPAN,TLAJOCOTLA (LA HACIENDA),Cultivo de maíz grano,27/06/2019,2430.55,PV2019,guerrero-teloloapan,12,Guerrero,58,Teloloapan
3,ROMERO GONZALEZ CELERINA,CENTRO PAÍS,GUERRERO,TELOLOAPAN,TELOLOAPAN,Cultivo de maíz grano,17/07/2019,4383.10,PV2019,guerrero-teloloapan,12,Guerrero,58,Teloloapan
4,GOMEZ CERON MA.MIREYA,CENTRO PAÍS,GUERRERO,TECOANAPA,LOS SAUCES,Cultivo de maíz grano,18/07/2019,4383.10,PV2019,guerrero-tecoanapa,12,Guerrero,56,Tecoanapa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275018,CASTILLO CUEVAS AMANCIO,CENTRO PAÍS,GUERRERO,TAXCO DE ALARCÓN,PAINTLA,Cultivo de maíz grano,27/06/2019,2430.55,PV2019,guerrero-taxco de alarcon,12,Guerrero,55,Taxco de Alarcón
275019,RUEDA SECUNDINO ANACLETO,CENTRO PAÍS,GUERRERO,TAXCO DE ALARCÓN,PAINTLA,Cultivo de maíz grano,27/06/2019,2430.55,PV2019,guerrero-taxco de alarcon,12,Guerrero,55,Taxco de Alarcón
275020,ARMENTA VILLEGAS LUIS,CENTRO PAÍS,GUERRERO,COCULA,COCULA,Cultivo de maíz grano,06/07/2019,2430.55,PV2019,guerrero-cocula,12,Guerrero,17,Cocula
275021,BELLO JIMENEZ GREGORIA,CENTRO PAÍS,GUERRERO,ATLIXTAC,XALPITZÁHUAC,Cultivo de maíz grano,15/07/2019,6574.65,PV2019,guerrero-atlixtac,12,Guerrero,10,Atlixtac


In [36]:
listado_beneficiarios_parte_II.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD', 'PRODUCTO',
       'FECHA', 'MONTO FEDERAL', 'CICLO AGRÍCOLA', 'KEY_benef_mun', 'CVE_ENT',
       'Entidad_inegi', 'CVE_MUN', 'Municipio_inegi'],
      dtype='object')

In [37]:
listado_beneficiarios_parte_II.to_csv('../../data/listados_completos/listado_beneficiarios_2019.csv', index=False)

In [38]:
listado_beneficiarios_parte_II.isna().sum()

BENEFICIARIO       0
ZONA               0
ENTIDAD            0
MUNICIPIO          0
LOCALIDAD          0
PRODUCTO           0
FECHA              0
MONTO FEDERAL      0
CICLO AGRÍCOLA     0
KEY_benef_mun      0
CVE_ENT            0
Entidad_inegi      0
CVE_MUN            0
Municipio_inegi    0
dtype: int64

# 3. Join por entidad (Divide y Vencerás)

In [39]:
listado_beneficiarios_2019_match = pd.read_csv('../../data/listados_completos/listado_beneficiarios_2019_match.csv')

In [40]:
listado_beneficiarios_2019_match.columns

Index(['Unnamed: 0', 'BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO',
       'LOCALIDAD', 'PRODUCTO', 'FECHA', 'MONTO FEDERAL', 'CICLO AGRÍCOLA'],
      dtype='object')

In [41]:
# Obtenemos las localidades únicas en el dataset.
listado_beneficiarios_2019_match = listado_beneficiarios_2019_match.drop(columns=['Unnamed: 0'])

In [42]:
listado_beneficiarios_2019_match['ENTIDAD_c_benef'] = listado_beneficiarios_2019_match['ENTIDAD'].apply(clean_text)
listado_beneficiarios_2019_match['MUNICIPIO_c_benef'] = listado_beneficiarios_2019_match['MUNICIPIO'].apply(clean_text)
listado_beneficiarios_2019_match['LOCALIDAD_c_benef'] = listado_beneficiarios_2019_match['LOCALIDAD'].apply(clean_text)

In [43]:
listado_beneficiarios_2019_match['KEY_benef_loc'] = listado_beneficiarios_2019_match['MUNICIPIO_c_benef'].astype(str) + '-' + listado_beneficiarios_2019_match['LOCALIDAD_c_benef'].astype(str)

In [44]:
listado_beneficiarios_2019_match.shape

(276554, 13)

In [45]:
listado_beneficiarios_2019_match = listado_beneficiarios_2019_match.drop_duplicates(subset=['KEY_benef_loc'])

In [46]:
listado_beneficiarios_2019_match.shape

(4658, 13)

In [47]:
listado_beneficiarios_2019_match.ENTIDAD.unique()

array(['Puebla', 'Guerrero', 'México', 'Guanajuato', 'Querétaro',
       'Zacatecas', 'Veracruz de Ignacio de la Llave', 'Hidalgo',
       'Michoacán de Ocampo', 'Oaxaca', 'Colima', 'Chiapas',
       'San Luis Potosí', 'Jalisco', 'Nayarit', 'GUERRERO'], dtype=object)

### 3.2.1 Join de localidades - GUERRERO

Empezamos creando los datasets por entiedad del dataset de beneficiarios.


In [48]:
entities = [
    'Puebla', 'México', 'Guanajuato', 'Querétaro',
    'Zacatecas', 'Veracruz de Ignacio de la Llave', 'Hidalgo',
    'Michoacán de Ocampo', 'Oaxaca', 'Colima', 'Chiapas',
    'San Luis Potosí', 'Jalisco', 'Nayarit', 'GUERRERO'
]

for entity in entities:
    variable_name = f"Localidades_19_{entity.upper().replace(' ', '_')}"
    dataframe = globals()[variable_name] = listado_beneficiarios_2019_match[listado_beneficiarios_2019_match["ENTIDAD"] == entity]
    print(f"{variable_name}: {dataframe.shape}")

Localidades_19_PUEBLA: (3, 13)
Localidades_19_MÉXICO: (4, 13)
Localidades_19_GUANAJUATO: (1, 13)
Localidades_19_QUERÉTARO: (1, 13)
Localidades_19_ZACATECAS: (4, 13)
Localidades_19_VERACRUZ_DE_IGNACIO_DE_LA_LLAVE: (1, 13)
Localidades_19_HIDALGO: (1, 13)
Localidades_19_MICHOACÁN_DE_OCAMPO: (1, 13)
Localidades_19_OAXACA: (5, 13)
Localidades_19_COLIMA: (1, 13)
Localidades_19_CHIAPAS: (2, 13)
Localidades_19_SAN_LUIS_POTOSÍ: (1, 13)
Localidades_19_JALISCO: (2, 13)
Localidades_19_NAYARIT: (1, 13)
Localidades_19_GUERRERO: (4576, 13)


In [49]:
Localidades_19_Guerrero = listado_beneficiarios_2019_match[listado_beneficiarios_2019_match['ENTIDAD'] == 'Guerrero']

In [50]:
Localidades_19_Guerrero.shape

(54, 13)

### 3.2.2 Diccionarios de Inegi por entidad.

In [51]:
folder_path = '../../data/productores_beneficiarios 2019-2022/diccionarios_E3/2019'

# Listar todos los archivos en la carpeta
files = os.listdir(folder_path)

# Filtrar solo los archivos CSV
csv_files = [file for file in files if file.endswith('.csv')]

# Leer cada archivo CSV y almacenar en variables separadas
for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    # Crear un nombre de variable dinámicamente eliminando la extensión .csv
    var_name = file_name.replace('.csv', '')
    # Leer el archivo CSV y asignar el DataFrame a la variable
    globals()[var_name] = pd.read_csv(file_path)

# Opcional: Imprimir el nombre de las variables creadas y la cantidad de filas y columnas de cada DataFrame
for file_name in csv_files:
    var_name = file_name.replace('.csv', '')
    df = globals()[var_name]
    print(f'{var_name}: {df.shape[0]} filas, {df.shape[1]} columnas')


INEGI_UNIQUELOC_2019_jalisco: 18306 filas, 11 columnas
INEGI_UNIQUELOC_2019_colima: 2647 filas, 11 columnas
INEGI_UNIQUELOC_2019_guanajuato: 11130 filas, 11 columnas
INEGI_UNIQUELOC_2019_nayarit: 4474 filas, 11 columnas
INEGI_UNIQUELOC_2019_veracruz_de_ignacio_de_la_llave: 25305 filas, 11 columnas
INEGI_UNIQUELOC_2019_zacatecas: 6418 filas, 11 columnas
INEGI_UNIQUELOC_2019_michoacán_de_ocampo: 11119 filas, 11 columnas
INEGI_UNIQUELOC_2019_hidalgo: 5279 filas, 11 columnas
INEGI_UNIQUELOC_2019_chiapas: 21887 filas, 11 columnas
INEGI_UNIQUELOC_2019_querétaro: 2829 filas, 11 columnas
INEGI_UNIQUELOC_2019_méxico: 5217 filas, 11 columnas
INEGI_UNIQUELOC_2019_oaxaca: 12820 filas, 11 columnas
INEGI_UNIQUELOC_2019_san_luis_potosí: 8946 filas, 11 columnas
INEGI_UNIQUELOC_2019_guerrero: 9194 filas, 11 columnas
INEGI_UNIQUELOC_2019_puebla: 7708 filas, 11 columnas


In [52]:
INEGI_UNIQUELOC_2019_guerrero.shape

(9194, 11)

In [53]:
Localidades_19_GUERRERO_parte_I = fuzzy_merge_benef2019_2022(Localidades_19_GUERRERO, INEGI_UNIQUELOC_2019_guerrero, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_GUERRERO_parte_I.shape

Localidades_19_PUEBLA_parte_I = fuzzy_merge_benef2019_2022(Localidades_19_PUEBLA, INEGI_UNIQUELOC_2019_puebla, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_PUEBLA_parte_I.shape

Localidades_19_MEXICO_parte_I = fuzzy_merge_benef2019_2022(Localidades_19_MÉXICO, INEGI_UNIQUELOC_2019_méxico, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_MEXICO_parte_I.shape

Localidades_19_GUANAJUATO_parte_I = fuzzy_merge_benef2019_2022(Localidades_19_GUANAJUATO, INEGI_UNIQUELOC_2019_guanajuato, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_GUANAJUATO_parte_I.shape

Localidades_19_QUERETARO_parte_I = fuzzy_merge_benef2019_2022(Localidades_19_QUERÉTARO, INEGI_UNIQUELOC_2019_querétaro, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_QUERETARO_parte_I.shape

Localidades_19_ZACATECAS_parte_I = fuzzy_merge_benef2019_2022(Localidades_19_ZACATECAS, INEGI_UNIQUELOC_2019_zacatecas, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_ZACATECAS_parte_I.shape

Localidades_19_VERACRUZ_DE_IGNACIO_DE_LA_LLAVE_parte_I = fuzzy_merge_benef2019_2022(Localidades_19_VERACRUZ_DE_IGNACIO_DE_LA_LLAVE, INEGI_UNIQUELOC_2019_veracruz_de_ignacio_de_la_llave, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_VERACRUZ_DE_IGNACIO_DE_LA_LLAVE_parte_I.shape

Localidades_19_HIDALGO_parte_I = fuzzy_merge_benef2019_2022(Localidades_19_HIDALGO, INEGI_UNIQUELOC_2019_hidalgo, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_HIDALGO_parte_I.shape

Localidades_19_MICHOACAN_parte_I = fuzzy_merge_benef2019_2022(Localidades_19_MICHOACÁN_DE_OCAMPO, INEGI_UNIQUELOC_2019_michoacán_de_ocampo, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_MICHOACAN_parte_I.shape

Localidades_19_OAXACA_parte_I = fuzzy_merge_benef2019_2022(Localidades_19_OAXACA, INEGI_UNIQUELOC_2019_oaxaca, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_OAXACA_parte_I.shape

Localidades_19_COLIMA_parte_I = fuzzy_merge_benef2019_2022(Localidades_19_COLIMA, INEGI_UNIQUELOC_2019_colima, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_COLIMA_parte_I.shape

Localidades_19_CHIAPAS_parte_I = fuzzy_merge_benef2019_2022(Localidades_19_CHIAPAS, INEGI_UNIQUELOC_2019_chiapas, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_CHIAPAS_parte_I.shape

Localidades_19_POTOSI_parte_I = fuzzy_merge_benef2019_2022(Localidades_19_SAN_LUIS_POTOSÍ, INEGI_UNIQUELOC_2019_san_luis_potosí, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_POTOSI_parte_I.shape

Localidades_19_JALISCO_parte_I = fuzzy_merge_benef2019_2022(Localidades_19_JALISCO, INEGI_UNIQUELOC_2019_jalisco, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_JALISCO_parte_I.shape

Localidades_19_NAYARIT_parte_I = fuzzy_merge_benef2019_2022(Localidades_19_NAYARIT, INEGI_UNIQUELOC_2019_nayarit, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_NAYARIT_parte_I.shape

/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_5007/734782849.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_benef['best_match'] = [match[0] if match else None for match in matches]
/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_5007/734782849.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_benef['match_score'] = [match[1] if match else None for match in matches]
/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_5007/734782849.py:18: SettingWithCopyWarning: 

(1, 26)

In [54]:
# Concatenate datasets vertically
diccionario_LOC_19_simple = pd.concat([
    Localidades_19_GUERRERO_parte_I,
    Localidades_19_PUEBLA_parte_I,
    Localidades_19_MEXICO_parte_I,
    Localidades_19_GUANAJUATO_parte_I,
    Localidades_19_QUERETARO_parte_I,
    Localidades_19_ZACATECAS_parte_I,
    Localidades_19_VERACRUZ_DE_IGNACIO_DE_LA_LLAVE_parte_I,
    Localidades_19_HIDALGO_parte_I,
    Localidades_19_MICHOACAN_parte_I,
    Localidades_19_OAXACA_parte_I,
    Localidades_19_COLIMA_parte_I,
    Localidades_19_CHIAPAS_parte_I,
    Localidades_19_POTOSI_parte_I,
    Localidades_19_JALISCO_parte_I,
    Localidades_19_NAYARIT_parte_I
], axis=0)

# Check the shape of the concatenated DataFrame
diccionario_LOC_19_simple.shape

(4604, 26)

In [55]:
diccionario_LOC_19_simple.drop(columns=['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD', 'PRODUCTO',
       'FECHA', 'MONTO FEDERAL', 'CICLO AGRÍCOLA', 'ENTIDAD_c_benef',
       'MUNICIPIO_c_benef', 'LOCALIDAD_c_benef', 'best_match', 'CVE_ENT', 'Entidad_inegi', 'CVE_MUN', 'Municipio_inegi',
       'CVE_LOC', 'Localidad_inegi', 'POB_TOTAL', 'Entidad_c_inegi',
       'Municipio_c_inegi', 'Localidad_c_inegi'], inplace=True)

In [56]:
diccionario_LOC_19_simple.to_csv('../../data/productores_beneficiarios 2019-2022/diccionarios_E3/diccionario_LOC_19_simple.csv', index=False)